In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

In [9]:
date_range_observed = pd.date_range(start="2015-01-01", end="2024-07-24 23:00:00", freq='H')
np.random.seed(42)
observed_data = np.random.randn(len(date_range_observed),3)  # Random normal data
df_observed = pd.DataFrame(observed_data, index=date_range_observed, columns=['wind','solar_pv','solar_th'])
date_range_predicted = pd.date_range(start=date_range_observed[-365*24], end=date_range_observed[-1], freq='H')
predicted_data = np.random.randn(len(date_range_predicted),3)  # Random normal data
df_predicted = pd.DataFrame(predicted_data, index=date_range_predicted, columns=['wind','solar_pv','solar_th'])
df_observed.head(), df_predicted.head()

/tmp/ipykernel_2766/1096351841.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range_observed = pd.date_range(start="2015-01-01", end="2024-07-24 23:00:00", freq='H')
/tmp/ipykernel_2766/1096351841.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range_predicted = pd.date_range(start=date_range_observed[-365*24], end=date_range_observed[-1], freq='H')


(                         wind  solar_pv  solar_th
 2015-01-01 00:00:00  0.496714 -0.138264  0.647689
 2015-01-01 01:00:00  1.523030 -0.234153 -0.234137
 2015-01-01 02:00:00  1.579213  0.767435 -0.469474
 2015-01-01 03:00:00  0.542560 -0.463418 -0.465730
 2015-01-01 04:00:00  0.241962 -1.913280 -1.724918,
                          wind  solar_pv  solar_th
 2023-07-26 00:00:00  1.092122 -0.513042  2.805864
 2023-07-26 01:00:00 -1.243588 -1.222445 -1.231983
 2023-07-26 02:00:00 -0.418960 -2.012888 -1.547864
 2023-07-26 03:00:00 -0.130661 -0.576877  1.279909
 2023-07-26 04:00:00  0.680455 -0.556472 -0.859357)

### Cramer Von Mises

In [3]:
from scipy.stats import cramervonmises, ecdf

In [11]:
results = {}
for c in df_observed.columns:
    cdf = ecdf(df_observed[c])
    cdf = cdf.cdf
    res = cramervonmises(df_predicted[c], cdf.evaluate)
    results[c] = res.statistic
results

{'wind': np.float64(0.1657046415792981),
 'solar_pv': np.float64(0.09824392885688041),
 'solar_th': np.float64(0.13005994966010606)}

### KL divergence

In [12]:
from scipy.stats import entropy
from sklearn.neighbors import KernelDensity

In [13]:
# n_bins = [30,60,90,150,300,500,1000]
# for num_bins in n_bins:
#     # num_bins = 30
#     observed_hist, bin_edges = np.histogram(df_observed['obs'], bins=num_bins, density=True)
#     predicted_hist, _ = np.histogram(df_predicted['predicted'], bins=bin_edges, density=True)
#     predicted_hist += 1e-10
#     kl_divergence = entropy(observed_hist, predicted_hist)
#     print(num_bins, kl_divergence)

In [14]:
results = {}
for c in df_observed.columns:
    # Fit KDE for observed and predicted data
    kde_observed = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(df_observed[c].values.reshape(-1, 1))
    kde_predicted = KernelDensity(kernel='gaussian', bandwidth=0.1).fit(df_predicted[c].values.reshape(-1, 1))

    # Evaluate KDE on a common set of points (a linear space covering both observed and predicted ranges)
    common_grid = np.linspace(min(df_observed[c].min(), df_predicted[c].min()),
                            max(df_observed[c].max(), df_predicted[c].max()), 5000).reshape(-1, 1)

    # Compute log density values
    log_density_observed = kde_observed.score_samples(common_grid)
    log_density_predicted = kde_predicted.score_samples(common_grid)

    # Convert log densities to actual densities
    density_observed = np.exp(log_density_observed)
    density_predicted = np.exp(log_density_predicted)

    # Add a small constant to prevent division by zero
    density_predicted += 1e-10

    # Calculate KL divergence
    kl_divergence_kde = entropy(density_observed, density_predicted)

    results[c]=kl_divergence_kde
results

{'wind': np.float64(0.001839808401045338),
 'solar_pv': np.float64(0.001806386172779419),
 'solar_th': np.float64(0.0014269050126308133)}

### ACF distance

In [23]:
def new_corr_coef(df, x='x', y='y'):
    """
    s_x: pd.Series, np.array
    """
    n = len(df)
    df = df.sort_values(by=x, ascending=True)
    rank = df[y].rank(method='min')
    coef = 1 - 3*sum(abs(rank-rank.shift(1))[1:])/(n**2-1)
    return coef

def calc_lcf(series_org, series_lag, n_lags=24*3):
    assert len(series_org) == len(series_lag)
    rhos = []
    xis = []
    for lag in range(n_lags + 1):
        shifted_series = series_lag.shift(lag)
        rho = series_org.corr(shifted_series)
        df = pd.DataFrame({'x':shifted_series, 'y':series_org})
        df = df.dropna()
        xi = new_corr_coef(df)
        rhos.append(rho)
        xis.append(xi)
    return rhos, xis

def calc_cf_dist(cf_obs,cf_pred):
    cf_obs = np.array(cf_obs)
    cf_pred = np.array(cf_pred)
    w = np.abs(cf_obs)
    return np.sqrt((w*(cf_obs-cf_pred)**2).sum()/w.sum())

lcs = {}
# lcs structure: {series: ((xi_obs, xi_pred),(rho_obs,rho_pred))}
for c in df_observed.columns:
    lc_rhos_pred, lc_xis_pred = calc_lcf(df_predicted[c],df_predicted[c])
    lc_rhos_obs, lc_xis_obs = calc_lcf(df_observed[c],df_observed[c])
    acfd_xi = calc_cf_dist(lc_xis_obs,lc_xis_pred)
    acfd_rho = calc_cf_dist(lc_rhos_obs, lc_rhos_pred)
    lcs[c] = (acfd_xi,acfd_rho)

print(lcs)


{'wind': (np.float64(0.002280553488101022), np.float64(0.004999741019547082)), 'solar_pv': (np.float64(0.0022352269308648694), np.float64(0.004578077734962516)), 'solar_th': (np.float64(0.002675986753910811), np.float64(0.005143137015705897))}


### CCMD

In [8]:
from copulas.univariate import GaussianKDE
from copulas.multivariate import GaussianMultivariate

In [17]:
def calc_mat_dist(m1,m2):
    n = len(m1)
    res = 0
    for i in range(1,n):
        for j in range(0,n-1):
            res += (m1[i][j] - m2[i][j])**2
    res /= (n*(n-1)/2)
    return res**.5

dist_obs = GaussianMultivariate(distribution={
    'solar_pv':GaussianKDE,
    'solar_th':GaussianKDE,
    'wind':GaussianKDE,
})
dist_obs.fit(df_observed.sample(7500,random_state=42))
params_obs = dist_obs.to_dict()
dist_pred = GaussianMultivariate(distribution={
    'solar_pv':GaussianKDE,
    'solar_th':GaussianKDE,
    'wind':GaussianKDE,
})
dist_pred.fit(df_predicted.sample(7500,random_state=42))
params_pred = dist_pred.to_dict()
ccmd = calc_mat_dist(params_obs['correlation'],params_pred['correlation'])
print(ccmd)

0.01208680919821136


### CCF distance